In [1]:
import json
import os
import google.generativeai as genai
import time

# --- Konfigurace ---
# Vložte svůj API klíč, který získáte z Google AI Studia.
# Klíč je doporučeno nastavit jako systémovou proměnnou pro vyšší bezpečnost.
# Například: GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
GOOGLE_API_KEY = "VLOZTE_SVUJ_API_KLIC_ZDE"

# Názvy vstupního a výstupního souboru
VSTUPNI_SOUBOR = 'otazky.json'
VYSTUPNI_SOUBOR = 'otazky_obohatene.json'

# --- Inicializace Gemini API ---
try:
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
except Exception as e:
    print(f"Chyba při konfiguraci Gemini API: {e}")
    print("Ujistěte se, že jste správně vložili svůj API klíč.")
    exit()

def analyzuj_otazku_pomoci_gemini(otazka_obj):
    """
    Funkce vezme objekt otázky, sestaví prompt pro Gemini,
    zavolá API a vrátí kategorii a vysvětlení.
    """
    kategorie = [
        "Letecké předpisy", "Lidská výkonnost", "Meteorologie", "Navigace",
        "Provozní postupy", "Letové výkony a plánování", "Znalosti letadel",
        "Principy letu", "Radiokomunikace"
    ]

    # Sestavení textu pro prompt
    text_otazky = otazka_obj['text_otazky']
    moznosti = otazka_obj['moznosti']
    spravna_odpoved_klic = otazka_obj['spravna_odpoved']
    spravna_odpoved_text = moznosti[spravna_odpoved_klic]

    prompt = f"""
    Analyzuj následující testovou otázku z oblasti letectví. Tvým úkolem je:
    1. Zařadit otázku do jedné z následujících kategorií: {', '.join(kategorie)}.
    2. Poskytnout stručné a jasné vysvětlení, proč je označená odpověď správná.

    Otázka: "{text_otazky}"
    Možnosti:
    A: "{moznosti.get('A', '')}"
    B: "{moznosti.get('B', '')}"
    C: "{moznosti.get('C', '')}"

    Správná odpověď je: {spravna_odpoved_klic} ("{spravna_odpoved_text}")

    Odpověz ve formátu JSON s klíči "kategorie" a "vysvetleni".
    Příklad odpovědi:
    {{
        "kategorie": "Principy letu",
        "vysvetleni": "Toto je ukázkové vysvětlení, proč je daná odpověď správná."
    }}
    """

    try:
        # Volání API
        response = model.generate_content(prompt)

        # Očištění a parsování odpovědi
        # API může odpověď zabalit do ```json ... ``` bloku
        cleaned_response = response.text.strip().replace('```json', '').replace('```', '').strip()

        data = json.loads(cleaned_response)
        return data['kategorie'], data['vysvetleni']
    except Exception as e:
        print(f"Nastala chyba při komunikaci s API pro otázku: '{text_otazky}'. Chyba: {e}")
        print(f"Odpověď od API, která způsobila chybu: {response.text if 'response' in locals() else 'Žádná odpověď'}")
        return "Neznámá kategorie", "Chyba při generování vysvětlení."

def zpracuj_otazky():
    """
    Hlavní funkce, která načte soubor, iteruje otázkami, volá API
    a ukládá výsledky.
    """
    try:
        with open(VSTUPNI_SOUBOR, 'r', encoding='utf-8') as f:
            otazky_data = json.load(f)
    except FileNotFoundError:
        print(f"Chyba: Vstupní soubor '{VSTUPNI_SOUBOR}' nebyl nalezen.")
        # Vytvoření ukázkového souboru pro testování
        print("Vytvářím ukázkový soubor 'otazky.json' pro demonstraci.")
        sample_data = [
            {
                "text_otazky": "Do jaké výšky lze počítat s „přízemním efektem“ v blízkosti země?",
                "moznosti": {
                    "A": "asi do výšky jako je polovina rozpětí křídla",
                    "B": "do výšky 1 m",
                    "C": "u ULLa do výšky přibližně 20 m"
                },
                "spravna_odpoved": "A",
                "body": 1,
                "pocet_vyskytu": 5445,
                "poslední_výskyt": "2025-09-08",
                "první_výskyt": "2025-04-15"
            }
        ]
        with open(VSTUPNI_SOUBOR, 'w', encoding='utf-8') as f:
            json.dump(sample_data, f, ensure_ascii=False, indent=4)
        otazky_data = sample_data

    obohatene_otazky = []
    pocet_otazek = len(otazky_data)

    print(f"Nalezeno {pocet_otazek} otázek. Zahajuji zpracování...")

    for i, otazka in enumerate(otazky_data):
        print(f"Zpracovávám otázku {i+1}/{pocet_otazek}: \"{otazka['text_otazky'][:50]}...\"")

        kategorie, vysvetleni = analyzuj_otazku_pomoci_gemini(otazka)

        # Přidání nových informací k původnímu objektu otázky
        otazka['kategorie'] = kategorie
        otazka['vysvetleni'] = vysvetleni

        obohatene_otazky.append(otazka)

        # Uložení průběžných výsledků po každé otázce
        with open(VYSTUPNI_SOUBOR, 'w', encoding='utf-8') as f:
            json.dump(obohatene_otazky, f, ensure_ascii=False, indent=4)

        # Přidání krátké pauzy, aby se předešlo překročení limitů API (např. 60 requestů za minutu)
        time.sleep(1)

    print("-" * 20)
    print(f"Zpracování dokončeno. Obohacené otázky byly uloženy do souboru '{VYSTUPNI_SOUBOR}'.")



zpracuj_otazky()

ModuleNotFoundError: No module named 'google.generativeai'